# Food Classification with CNN - Building a Restaurant Recommendation System

This assignment focuses on developing a deep learning-based food classification system using Convolutional Neural Networks (CNNs). You will build a model that can recognize different food categories and use it to return the food preferences of a user.

## Learning Objectives
- Implement CNNs for image classification
- Work with real-world food image datasets
- Build a preference-detector system

## Background: AI-Powered Food Preference Discovery

The system's core idea is simple:

1. Users upload 10 photos of dishes they enjoy
2. Your CNN classifies these images into the 91 categories
3. Based on these categories, the system returns the user's taste profile

Your task is to develop the core computer vision component that will power this detection engine.

You are given a training ("train" folder) and a test ("test" folder) dataset which have ~45k and ~22k samples respectively. For each one of the 91 classes there is a subdirectory containing the images of the respective class.

## Assignment Requirements

### Technical Requirements
- Implement your own pytorch CNN architecture for food image classification
- Use only the provided training dataset split for training
- Train the network from scratch ; No pretrained weights can be used
- Report test-accuracy after every epoch
- Report all hyperparameters of final model
- Use a fixed seed and do not use any CUDA-features that break reproducibility
- Use Pytorch 2.6

### Deliverables
1. Jupyter Notebook with CNN implementation, training code etc.
2. README file
3. Report (max 3 pages)

Submit your report, README and all code files as a single zip file named GROUP_[number]_NC2425_PA. The names and IDs of the group components must be mentioned in the README.
Do not include the dataset in your submission.

### Grading

1. Correct CNN implementation, training runs on the uni computers (computer rooms DM.0.07, DM.0.13, DM.0.17, DM.0.21) according to the README.MD instructions without ANY exceptions: 3pt
2. Perfect 1:1 reproducibility on those machines: 1pt
3. Very clear github-repo-style README.MD with instructions for running the code: 1pt
4. Report: 1pt
5. Model test performance on test-set: interpolated from 30-80% test-accuracy: 0-3pt
6. Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt
7. Bonus point: use an LLM (API) to generate short description / profile of preferences of the simulated user

**The main reason why we mention the machines in the uni computer rooms is to make it clear to you how we will test your submissions and to make you aware that you can use these machines for working on your assignment if you do not have access to a computer with enough gpu. You do not have to use these machines for this assignment. Using the specified pytorch version, not enabling torch.backends.cuda.matmul.allow_tf32, setting fixed seeds and making sure your code doesn't use more than 8gb VRAM should allow you to fulfill the reproducibility criteria.**

**(If there is anything unclear about this assignment please post your question in the Brightspace discussions forum or send an email)**


In [1]:
import pandas
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import ToTensor
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader, Dataset


In [90]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"   # deterministic mode for GPU 

random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)


# Loading the datasets
The dataset is already split into a train and test set in the directories "train" and "test". 

In [69]:
def img_to_tensor(path):
    """
    Transforms image into standardized grayscale and normalized tensor
    :param path: path to image
    :return: tensor
    """
    img = Image.open(path).convert("RGB")      # use color too
    img = img.resize((224, 224))
    arr = np.array(img, dtype=np.float32) / 255
    x = torch.tensor(arr.transpose(2, 0, 1))   # (C,H,W)
    return x

# make dataloaders out of train and test splits to be able to pass them to the CNN
# ---

splits = {'train': None, 'test': None} # dictionary {directory name: DataLoader} for the train test split
class_count = 0
for split in splits.keys():
    print(f"Making Dataloader for {split} set...")
    images = []
    ordered_labels = []
    for folder in os.scandir(split): # iterate through the contents of the directory
        if not folder.is_dir(): # make sure content is directory (could be .DS_Store file)
            continue
        class_count += 1
        for image in os.scandir(folder):
            ordered_labels.append(folder.name) # an image's parent folder name is its label
            image_tensor = img_to_tensor(image)
            images.append(image_tensor)

    
    # encode labels into numbers then convert label list to tensor
    encoder = LabelEncoder()
    ordered_labels = encoder.fit_transform(ordered_labels)
    ordered_labels_tensor = torch.LongTensor(ordered_labels)

    # make one tensor of all image tensors (like an array of images)
    all_image_tensor = torch.stack(images)

    # make torch dataloader out of images and labels tensors
    dataset = TensorDataset(all_image_tensor, ordered_labels_tensor)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    splits[split] = dataloader
print("success")
# ---
train_loader = splits['train']
test_loader  = splits['test']

Making Dataloader for train set...
Making Dataloader for test set...
success


In [49]:
# different dataloader approach to allow for data augmentation

class FoodDataset(Dataset):
    def __init__(self, root_dir, transform=None, encoder=None):
        self.images = []
        self.labels = []
        self.transform = transform
        self.encoder = encoder

        string_labels = []
        for folder in os.scandir(root_dir): # iterate through the contents of the directory
            if not folder.is_dir(): # make sure content is directory (could be .DS_Store file)
                continue
            for image in os.scandir(folder):
                string_labels.append(folder.name) # an image's parent folder name is its label
                self.images.append(image)
        self.labels = self.encoder.transform(string_labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

    def get_normalization_values(self):
        pixel_sum = np.zeros(3)
        pixel_squared_sum = np.zeros(3)
        pixel_count = 0
        for image in self.images:
            img = Image.open(image).convert("RGB").resize((224, 224))
            img = np.array(img).astype(np.float32) / 255.0  # shape: (H, W, C)
            pixel_sum += img.sum(axis=(0, 1))
            pixel_squared_sum += (img ** 2).sum(axis=(0, 1))
            pixel_count += img.shape[0] * img.shape[1]

        mean_rgb = pixel_sum / pixel_count
        std_rgb = np.sqrt(pixel_squared_sum / pixel_count - mean_rgb ** 2)
        return mean_rgb, std_rgb

    def set_transform(self, transform):
        self.transform = transform


class_names = []
for folder in os.scandir("train"):
    if folder.is_dir():
        class_names.append(folder.name)
encoder = LabelEncoder()
encoder.fit(class_names)
labels = encoder.fit_transform(class_names)

print("making train_dataset")
train_dataset = FoodDataset(root_dir="train", encoder=encoder)
print("making test_dataset\n")
test_dataset  = FoodDataset(root_dir="test", encoder=encoder)

print("calculating train normalization values")
train_mean, train_std = train_dataset.get_normalization_values()
print(f"mean = {train_mean}, std = {train_std}\n")

train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(train_mean, train_std)
])

print("calculating test normalization values:")
test_mean, test_std = test_dataset.get_normalization_values()
print(f"mean = {train_mean}, std = {train_std}\n")

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(test_mean, test_std)
])

print("setting transforms")
train_dataset.set_transform(train_transforms)
test_dataset.set_transform(test_transforms)

print("making dataloaders")
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)
print("done")

# test dataset
x, y = next(iter(train_loader))
print("Input shape:", x.shape)  # Expect: torch.Size([16, 3, 224, 224])
print("Input range:", x.min().item(), "to", x.max().item())  # Expect: 0–1 or ~-2 to 2
print("Labels:", y[:10])  # Expect: LongTensor of class indices 0–90
print("Label dtype:", y.dtype)  # Expect: torch.int64

making train_dataset
making test_dataset

making dataloaders
done
Input shape: torch.Size([16, 3, 224, 224])
Input range: 0.0 to 1.0
Labels: tensor([32, 79, 46, 59,  0, 38, 39, 59,  6, 46])
Label dtype: torch.int64


In [70]:
# small scale dataset for test
from torch.utils.data import Subset

subset = Subset(train_dataset, list(range(32)))
tiny_loader = DataLoader(subset, batch_size=8, shuffle=True)

# CNN Implementation

In [85]:
# CNN taken from practical on brightspace
# needs further tweaking
def get_num_classes(f):
    class_count = 0
    for folder in os.scandir(f): # iterate through the contents of the directory
        if not folder.is_dir(): # make sure content is directory (could be .DS_Store file)
            continue
        class_count += 1
    return class_count
batch_size = 16 # 16: safe, 32: faster but might not run on every pc
num_classes = get_num_classes("train")
learning_rate = 0.001
num_epochs = 20

class FoodCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.MaxPool2d(2, 2),

        )

        # Dynamically compute flattened size
        dummy_input = torch.zeros(1, 3, 224, 224)
        flat_size = self.features(dummy_input).numel()
        print("flat_size:", flat_size)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flat_size, 1024),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Linear(1024, 512),
            nn.ReLU(),
            # nn.Dropout(0.3),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


91


In [87]:
# original cnn
def get_num_classes(f):
    class_count = 0
    for folder in os.scandir(f): # iterate through the contents of the directory
        if not folder.is_dir(): # make sure content is directory (could be .DS_Store file)
            continue
        class_count += 1
    return class_count
batch_size = 16 # 16: safe, 32: faster but might not run on every pc
num_classes = get_num_classes("train")
learning_rate = 0.001
num_epochs = 20

class FoodCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),  # use RGB → in_channels=3
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                 # 3×224×224 → 32×112×112

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                 # 64×56×56
        )

        # compute flattened size once
        dummy = torch.zeros(1, 3, 224, 224)
        flat = self.features(dummy).numel()

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flat, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Training the model
Implement your training process below. Report the test-accuracy after every epoch for the training run of the final model.

Hint: before training your model make sure to reset the seed in the training cell, as otherwise the seed may have changed due to previous training runs in the notebook

Note: If you implement automatic hyperparameter tuning, split the train set into train and validation subsets for the objective function.

In [88]:
# check if GPU is available and use it if possible, run on CPU otherwise

import torch
print(torch.__version__, torch.version.cuda)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


2.7.0+cu128 12.8
True
NVIDIA GeForce RTX 4070 Laptop GPU


In [89]:
     # reset the seed in this cell
def accuracy(logits, labels):
    preds = logits.argmax(dim=1)
    return (preds == labels).float().mean().item() * 100

def evaluate(model, loader):
    model.eval()
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            acc_sum += accuracy(model(x), y) * x.size(0)
            n += x.size(0)

            preds = model(x).argmax(dim=1)
            print("Sample predictions:", preds[:5].tolist())
            print("As labels:", encoder.inverse_transform(preds[:5].tolist()))

    return acc_sum / n


device = "cuda" if torch.cuda.is_available() else "cpu"
model = FoodCNN(num_classes).to(device)

criterion  = nn.CrossEntropyLoss()
optimizer  = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler  = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

best = 0.0
for epoch in range(1, 21):
    model.train()
    print("training epoch:", epoch)
    running_loss = 0.0
    for x, y in train_loader:

        x, y = x.to(device), y.to(device)

        loss = criterion(model(x), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    test_acc = evaluate(model, test_loader)
    print(f"epoch {epoch:02d} | test acc {test_acc:.2f}%")
    print(f"Epoch {epoch:02d} | Train loss: {running_loss:.4f}")

    if test_acc > best:
        torch.save(model.state_dict(), "best_model.pth")
        best = test_acc

    scheduler.step()


training epoch: 1
Sample predictions: [11, 11, 11, 11, 11]
As labels: ['cheese_plate' 'cheese_plate' 'cheese_plate' 'cheese_plate'
 'cheese_plate']
Sample predictions: [11, 11, 11, 11, 11]
As labels: ['cheese_plate' 'cheese_plate' 'cheese_plate' 'cheese_plate'
 'cheese_plate']
Sample predictions: [11, 11, 11, 11, 11]
As labels: ['cheese_plate' 'cheese_plate' 'cheese_plate' 'cheese_plate'
 'cheese_plate']
Sample predictions: [11, 11, 11, 11, 11]
As labels: ['cheese_plate' 'cheese_plate' 'cheese_plate' 'cheese_plate'
 'cheese_plate']
Sample predictions: [11, 11, 11, 11, 11]
As labels: ['cheese_plate' 'cheese_plate' 'cheese_plate' 'cheese_plate'
 'cheese_plate']
Sample predictions: [11, 11, 11, 11, 11]
As labels: ['cheese_plate' 'cheese_plate' 'cheese_plate' 'cheese_plate'
 'cheese_plate']
Sample predictions: [11, 11, 11, 11, 11]
As labels: ['cheese_plate' 'cheese_plate' 'cheese_plate' 'cheese_plate'
 'cheese_plate']
Sample predictions: [11, 11, 11, 11, 11]
As labels: ['cheese_plate' 'che

KeyboardInterrupt: 

# Calculating model performance
Load the best version of your model ( which should be produced and saved by previous cells ), calculate and report the test accuracy.

In [ ]:
# Load the best model weights
model = FoodCNN(num_classes).to(device)
model.load_state_dict(torch.load("best_model.pth"))

final_test_acc = evaluate(model, test_loader)
print(f"Final Test Accuracy: {final_test_acc:.2f}%")


RuntimeError: Invalid device string: 'best_model.pth'

# Summary of hyperparameters
Report the hyperparameters ( learning rate etc ) that you used in your final model for reproducibility.

# Simulation of random user
Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt

In [ ]:
def get_random_images(folder):
    img_tensor_list = []
    classes_list = []
    for i in range(10):
        random_class = None
        while random_class is None:
            random_class = os.listdir(folder)[np.random.randint(0, len(folder) - 1)]
            if random_class == ".DS_Store": random_class = None
        classes_list.append(random_class)
        random_image = os.listdir(random_class)[np.random.randint(0, len(random_class) - 1)]
        img_tensor = img_to_tensor(random_image)
        img_tensor_list.append(img_tensor)
    all_img_tensor = torch.stack(img_tensor_list)
    return all_img_tensor, classes_list

def get_predictions(model, images, encoder):
    predictions = model(images)
    string_predictions = list(encoder.inverse_transform(predictions))
    return string_predictions

def make_frequency_dict(classes):
    freq_dict = {}
    for c in classes:
        if c not in freq_dict:
            freq_dict[c] = 1
        else:
            freq_dict[c] += 1
    return freq_dict

images, random_classes = get_random_images('train')
predictions = get_predictions(model, images, encoder)
actual_frequencies = make_frequency_dict(random_classes)
pred_frequencies = make_frequency_dict(predictions)

print(f"predicted class frequencies: {pred_frequencies}")
print(f"actual class frequencies: {actual_frequencies}")

# Below an example showing the format of the code output

# Bonus point
Use an LLM (API) to generate a description of the food preference of a user based on 10 images that a potential user could provide. 
Please include an example of the output of your code, especially if you used an API other than the OpenAI API.

This should work well even with differing test images by setting different random seeds for the image selector.